In [ ]:
# 🚀 BENCHMARK COLAB - Iperparametri Significativi
# Configurazione ottimizzata per Google Colab con ~50 unità di calcolo

import os
import json
import time
from datetime import datetime

# Configurazione benchmark per Colab
COLAB_BENCHMARK_CONFIG = {
    # Dataset ridotto per velocità
    "sample_size": 8000,  # Bilanciato tra velocità e significatività
    
    # Finestre temporali significative per cybersecurity
    "time_windows": [
        {"window_size": 30, "step": 15, "name": "30s_15s"},    # Attacchi rapidi
        {"window_size": 60, "step": 30, "name": "1m_30s"},     # Sessioni brevi
        {"window_size": 300, "step": 150, "name": "5m_2.5m"},  # Sessioni lunghe
    ],
    
    # Architetture da testare
    "models": [
        {
            "type": "dense", 
            "name": "Dense_Baseline",
            "params": {"epochs": 3, "batch_size": 64}
        },
        {
            "type": "gru", 
            "name": "GRU_Fast",
            "params": {"epochs": 3, "batch_size": 32, "gru_units": 64}
        },
        {
            "type": "lstm", 
            "name": "LSTM_Optimized",
            "params": {"epochs": 3, "batch_size": 32, "lstm_units": 64}
        }
    ],
    
    # Learning rates significativi
    "learning_rates": [0.001, 0.0005],  # Solo i più promettenti
    
    # Aggregazioni statistiche
    "aggregation_stats": [
        ["sum", "mean", "max"],           # Base veloce
        ["sum", "mean", "std", "max"],   # Completa
    ]
}

print("🎯 BENCHMARK COLAB CONFIGURATO")
print(f"📊 Sample size: {COLAB_BENCHMARK_CONFIG['sample_size']:,}")
print(f"⏱️ Finestre temporali: {len(COLAB_BENCHMARK_CONFIG['time_windows'])}")
print(f"🤖 Modelli: {len(COLAB_BENCHMARK_CONFIG['models'])}")
print(f"📈 Learning rates: {COLAB_BENCHMARK_CONFIG['learning_rates']}")
print(f"📊 Configurazioni aggregazione: {len(COLAB_BENCHMARK_CONFIG['aggregation_stats'])}")

# Stima tempo totale
total_configs = (len(COLAB_BENCHMARK_CONFIG['time_windows']) * 
                len(COLAB_BENCHMARK_CONFIG['models']) * 
                len(COLAB_BENCHMARK_CONFIG['learning_rates']) * 
                len(COLAB_BENCHMARK_CONFIG['aggregation_stats']))

print(f"🔢 Configurazioni totali: {total_configs}")
print(f"⏰ Tempo stimato: ~{total_configs * 2:.0f}-{total_configs * 4:.0f} minuti")


In [ ]:
# 🔧 Funzione di Benchmark Automatizzato

def run_colab_benchmark():
    """
    Esegue benchmark completo con configurazioni ottimizzate per Colab.
    Testa combinazioni significative di iperparametri.
    """
    
    # Import necessari
    from preprocessing.process import preprocess_pipeline
    from training.train import train_model
    from evaluation.evaluate import evaluate_model_comprehensive
    from config import PREPROCESSING_CONFIG, TRAINING_CONFIG
    
    # Risultati
    all_results = []
    start_time = time.time()
    
    print("🚀 AVVIO BENCHMARK COLAB")
    print("=" * 60)
    
    config_num = 0
    total_configs = (len(COLAB_BENCHMARK_CONFIG['time_windows']) * 
                    len(COLAB_BENCHMARK_CONFIG['models']) * 
                    len(COLAB_BENCHMARK_CONFIG['learning_rates']) * 
                    len(COLAB_BENCHMARK_CONFIG['aggregation_stats']))
    
    # Loop attraverso tutte le configurazioni
    for window_config in COLAB_BENCHMARK_CONFIG['time_windows']:
        for model_config in COLAB_BENCHMARK_CONFIG['models']:
            for lr in COLAB_BENCHMARK_CONFIG['learning_rates']:
                for agg_stats in COLAB_BENCHMARK_CONFIG['aggregation_stats']:
                    
                    config_num += 1
                    config_start = time.time()
                    
                    print(f"\n🧪 CONFIGURAZIONE {config_num}/{total_configs}")
                    print(f"⏱️ Finestra: {window_config['name']}")
                    print(f"🤖 Modello: {model_config['name']}")
                    print(f"📈 Learning Rate: {lr}")
                    print(f"📊 Aggregazioni: {agg_stats}")
                    print("-" * 40)
                    
                    try:
                        # 1. Configura preprocessing
                        PREPROCESSING_CONFIG.update({
                            'sample_size': COLAB_BENCHMARK_CONFIG['sample_size'],
                            'window_size': window_config['window_size'],
                            'step': window_config['step'],
                            'aggregation_stats': agg_stats
                        })
                        
                        # 2. Configura training
                        TRAINING_CONFIG.update({
                            'model_type': model_config['type'],
                            'hyperparameters': {
                                **model_config['params'],
                                'learning_rate': [lr]
                            }
                        })
                        
                        # 3. Preprocessing
                        print("🔄 Preprocessing...")
                        X, y, label_encoder = preprocess_pipeline()
                        
                        if X.shape[0] == 0:
                            print("❌ Nessun dato dopo preprocessing")
                            continue
                        
                        # 4. Training
                        print("🏋️ Training...")
                        model, training_log, model_path = train_model(
                            X, y, 
                            model_type=model_config['type']
                        )
                        
                        # 5. Valutazione rapida
                        print("📊 Valutazione...")
                        from sklearn.model_selection import train_test_split
                        X_train, X_test, y_train, y_test = train_test_split(
                            X, y, test_size=0.2, random_state=42, stratify=y
                        )
                        
                        eval_results = evaluate_model_comprehensive(
                            model, X_test, y_test, 
                            class_names=label_encoder.classes_.tolist(),
                            output_dir=f'colab_eval/config_{config_num}'
                        )\n                        
                        # 6. Salva risultati
                        config_time = time.time() - config_start
                        result = {
                            'config_id': config_num,
                            'window_config': window_config,
                            'model_config': model_config,
                            'learning_rate': lr,
                            'aggregation_stats': agg_stats,
                            'data_shape': {'X': X.shape, 'y': y.shape},
                            'accuracy': eval_results['basic_metrics']['accuracy'],
                            'f1_macro': eval_results['basic_metrics']['f1_macro'],
                            'training_time': training_log.get('training_time', 0),
                            'config_time': config_time,
                            'model_path': model_path
                        }
                        
                        all_results.append(result)
                        
                        print(f"✅ Completato in {config_time:.1f}s")
                        print(f"📊 Accuracy: {result['accuracy']:.3f}")
                        print(f"📊 F1-Macro: {result['f1_macro']:.3f}")
                        
                    except Exception as e:
                        print(f"❌ Errore configurazione {config_num}: {e}")
                        continue
    
    # Risultati finali
    total_time = time.time() - start_time
    print(f"\n🎉 BENCHMARK COMPLETATO!")
    print(f"⏰ Tempo totale: {total_time/60:.1f} minuti")
    print(f"✅ Configurazioni completate: {len(all_results)}/{total_configs}")
    
    # Salva risultati
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_file = f"colab_benchmark_results_{timestamp}.json"
    
    with open(results_file, 'w') as f:
        json.dump({
            'timestamp': timestamp,
            'total_time': total_time,
            'total_configs': total_configs,
            'completed_configs': len(all_results),
            'results': all_results
        }, f, indent=2, default=str)
    
    print(f"💾 Risultati salvati: {results_file}")
    
    return all_results, results_file

# Preparazione per esecuzione
print("🎯 Funzione benchmark pronta!")
print("💡 Per eseguire: all_results, results_file = run_colab_benchmark()")


In [ ]:
# 📊 Analisi e Visualizzazione Risultati

def analyze_benchmark_results(results_file):
    """
    Analizza i risultati del benchmark e crea visualizzazioni.
    """
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # Carica risultati
    with open(results_file, 'r') as f:
        data = json.load(f)
    
    results = data['results']
    df = pd.DataFrame(results)
    
    print(f"📊 ANALISI RISULTATI BENCHMARK")
    print(f"⏰ Tempo totale: {data['total_time']/60:.1f} minuti")
    print(f"✅ Configurazioni completate: {data['completed_configs']}/{data['total_configs']}")
    print("=" * 50)
    
    # 1. Migliori configurazioni
    print("\n🏆 TOP 5 CONFIGURAZIONI (Accuracy)")
    top_configs = df.nlargest(5, 'accuracy')[['config_id', 'accuracy', 'f1_macro', 'window_config', 'model_config']]
    for _, row in top_configs.iterrows():
        print(f"  Config {row['config_id']}: {row['accuracy']:.3f} acc, {row['f1_macro']:.3f} f1")
        print(f"    🪟 {row['window_config']['name']} | 🤖 {row['model_config']['name']}")
    
    # 2. Analisi per finestra temporale
    print("\n⏱️ PERFORMANCE PER FINESTRA TEMPORALE")
    window_stats = df.groupby(df['window_config'].apply(lambda x: x['name'])).agg({
        'accuracy': ['mean', 'std', 'max'],
        'f1_macro': ['mean', 'std', 'max'],
        'config_time': 'mean'
    }).round(3)
    print(window_stats)
    
    # 3. Analisi per modello
    print("\n🤖 PERFORMANCE PER MODELLO")
    model_stats = df.groupby(df['model_config'].apply(lambda x: x['name'])).agg({
        'accuracy': ['mean', 'std', 'max'],
        'f1_macro': ['mean', 'std', 'max'],
        'config_time': 'mean'
    }).round(3)
    print(model_stats)
    
    # 4. Visualizzazioni
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Accuracy per finestra
    window_names = [x['name'] for x in df['window_config']]
    df['window_name'] = window_names
    sns.boxplot(data=df, x='window_name', y='accuracy', ax=axes[0,0])
    axes[0,0].set_title('Accuracy per Finestra Temporale')
    axes[0,0].tick_params(axis='x', rotation=45)
    
    # Accuracy per modello
    model_names = [x['name'] for x in df['model_config']]
    df['model_name'] = model_names
    sns.boxplot(data=df, x='model_name', y='accuracy', ax=axes[0,1])
    axes[0,1].set_title('Accuracy per Modello')
    axes[0,1].tick_params(axis='x', rotation=45)
    
    # Tempo vs Accuracy
    axes[1,0].scatter(df['config_time'], df['accuracy'], alpha=0.7)
    axes[1,0].set_xlabel('Tempo Configurazione (s)')
    axes[1,0].set_ylabel('Accuracy')
    axes[1,0].set_title('Tempo vs Accuracy')
    
    # Learning Rate vs Accuracy
    sns.boxplot(data=df, x='learning_rate', y='accuracy', ax=axes[1,1])
    axes[1,1].set_title('Learning Rate vs Accuracy')
    
    plt.tight_layout()
    plt.savefig(f'benchmark_analysis_{data["timestamp"]}.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # 5. Raccomandazioni
    print("\n💡 RACCOMANDAZIONI")
    best_config = df.loc[df['accuracy'].idxmax()]
    print(f"🥇 Migliore configurazione: Config {best_config['config_id']}")
    print(f"   📊 Accuracy: {best_config['accuracy']:.3f}")
    print(f"   🪟 Finestra: {best_config['window_config']['name']}")
    print(f"   🤖 Modello: {best_config['model_config']['name']}")
    print(f"   📈 Learning Rate: {best_config['learning_rate']}")
    
    # Efficienza (accuracy/tempo)
    df['efficiency'] = df['accuracy'] / df['config_time']
    most_efficient = df.loc[df['efficiency'].idxmax()]
    print(f"\n⚡ Configurazione più efficiente: Config {most_efficient['config_id']}")
    print(f"   📊 Accuracy: {most_efficient['accuracy']:.3f}")
    print(f"   ⏱️ Tempo: {most_efficient['config_time']:.1f}s")
    print(f"   🎯 Efficienza: {most_efficient['efficiency']:.4f} acc/s")
    
    return df

print("📊 Funzione di analisi pronta!")
print("💡 Dopo il benchmark: df = analyze_benchmark_results(results_file)")


In [ ]:
# 🚀 ESECUZIONE BENCHMARK COLAB
# Esegui questa cella per avviare il benchmark completo

print("🎯 AVVIO BENCHMARK OTTIMIZZATO PER COLAB")
print("⚡ Configurazione: 36 test totali (~60-120 minuti)")
print("🔥 GPU consigliata per velocità ottimale")
print("\n" + "="*60)

# Esegui il benchmark
all_results, results_file = run_colab_benchmark()

print(f"\n🎉 BENCHMARK COMPLETATO!")
print(f"📁 File risultati: {results_file}")
print(f"📊 Configurazioni testate: {len(all_results)}")

# Analisi automatica
print("\n🔍 AVVIO ANALISI RISULTATI...")
df_results = analyze_benchmark_results(results_file)

print(f"\n✅ TUTTO COMPLETATO!")
print(f"💾 Risultati salvati in: {results_file}")
print(f"📈 Grafici salvati come: benchmark_analysis_*.png")


In [ ]:
# 🔧 Test Rapido di una Singola Configurazione
# Usa questa cella per testare rapidamente una configurazione specifica

def quick_test_config(window_size=60, step=30, model_type='gru', learning_rate=0.001):
    """
    Test rapido di una configurazione specifica.
    """
    from preprocessing.process import preprocess_pipeline
    from training.train import train_model
    from evaluation.evaluate import evaluate_model_comprehensive
    from config import PREPROCESSING_CONFIG, TRAINING_CONFIG
    from sklearn.model_selection import train_test_split
    
    print(f"🧪 TEST RAPIDO")
    print(f"⏱️ Finestra: {window_size}s (step: {step}s)")
    print(f"🤖 Modello: {model_type}")
    print(f"📈 Learning Rate: {learning_rate}")
    print("-" * 40)
    
    start_time = time.time()
    
    try:
        # Configura
        PREPROCESSING_CONFIG.update({
            'sample_size': 3000,  # Ridotto per velocità
            'window_size': window_size,
            'step': step,
            'aggregation_stats': ["sum", "mean", "max"]
        })
        
        TRAINING_CONFIG.update({
            'model_type': model_type,
            'hyperparameters': {
                'epochs': [2],  # Ridotto per velocità
                'batch_size': [32],
                'learning_rate': [learning_rate]
            }
        })
        
        # Pipeline
        print("🔄 Preprocessing...")
        X, y, label_encoder = preprocess_pipeline()
        
        print("🏋️ Training...")
        model, training_log, model_path = train_model(X, y, model_type=model_type)
        
        print("📊 Valutazione...")
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        eval_results = evaluate_model_comprehensive(
            model, X_test, y_test, 
            class_names=label_encoder.classes_.tolist(),
            output_dir='quick_test_eval'
        )
        
        total_time = time.time() - start_time
        
        print(f"\n✅ TEST COMPLETATO in {total_time:.1f}s")
        print(f"📊 Accuracy: {eval_results['basic_metrics']['accuracy']:.3f}")
        print(f"📊 F1-Macro: {eval_results['basic_metrics']['f1_macro']:.3f}")
        print(f"📊 Shape dati: X={X.shape}, y={y.shape}")
        
        return eval_results
        
    except Exception as e:
        print(f"❌ Errore: {e}")
        return None

print("🧪 Funzione test rapido pronta!")
print("💡 Esempio: quick_test_config(window_size=30, model_type='dense')")


## IDS Pipeline Audit: Preprocessing avanzato e Training

Questa sezione documenta dettagliatamente le scelte progettuali e operative per:
- **Flow reassembly** e **sessionizzazione** (timeout configurabile)
- **Finestre N/T** attorno al primo evento malevolo per `Flow_ID`
- **Label propagation** con modalità configurabile (any/majority/probabilistica/smoothing)
- **Noise handling** tramite majority/temporal smoothing
- **Dataset balancing** a livello di flusso (undersampling/SMOTE)
- **Output** per modelli sequenziali e MLP aggregato
- **Supporto training** con MLP 4 layer e logging delle loss

Tutte le funzioni sono eseguibili da **comandi**, per favorire riproducibilità e tracciabilità.


### Struttura della pipeline (overview)

- **Input**: CSV con campi `Flow_ID`, `Timestamp`, `Label` (es. `BENIGN` o altro)
- **Reassembly & Sessionizzazione**: raggruppa per `Flow_ID` e separa in `Session_ID` con timeout (default 60s)
- **Noise filtering**: smoothing/majority sulle etichette temporali
- **Finestra N/T**: per ogni `Session_ID`, crea una finestra centrata sul primo evento malevolo con `N` secondi prima e `T` dopo
- **Aggregazione**: in bin temporali (e.g., 5s) per sequenze; oppure aggregazione globale per MLP
- **Label propagation**: se la finestra contiene almeno un malevolo (o secondo strategia), etichetta la finestra come malevola
- **Bilanciamento**: seleziona tutte le sessioni malevole e un ugual numero di `BENIGN` (undersample) o usa SMOTE
- **Output**: salva `X.npy` e `y.npy` o shard per-epoca per training


### Parametri configurabili (config.py)

- **Dataset**:
  - `DATA_CONFIG.dataset_path`, `timestamp_column`, `flow_id_column`, `target_column`, `benign_label`, `timestamp_format`
- **Sessionizzazione**:
  - `PREPROCESSING_CONFIG.session_timeout_seconds`
- **Finestre**:
  - `flow_window_strategy` (e.g., `first_malicious_context`)
  - `window_before_first_malicious_s` (N), `window_after_first_malicious_s` (T)
  - `time_bin_seconds` (granularità per sequenze)
- **Label propagation / Noise**:
  - `label_propagation.mode` (`any`/`majority`/`probabilistic`/`smoothing`)
  - `prob_threshold`, `smoothing_alpha`
  - `noise_filter.enabled`, `noise_filter.method`, `noise_filter.window`, `noise_filter.threshold`
- **Bilanciamento a flusso**:
  - `flow_balance.enabled`, `flow_balance.method` (`undersample`/`smote`/`none`), `flow_balance.ratio`
- **Output**:
  - `output_mode` (`sequence`/`mlp_aggregated`), `aggregation_stats`
- **Training**:
  - `model_type`, `mlp_hidden_layers`, `dropout_rate`, `hyperparameters.epochs/batch_size/learning_rate`, `max_epochs`
  - `preprocess_per_epoch`, `flows_per_epoch`, `epoch_selection_mode`


### Esecuzione preprocessing (solo comandi)

1) Installare dipendenze
```bash
pip install -r requirements.txt
```

2) Verifica/adegua `config.py` (path dataset, colonne, parametri)

3) Esegui preprocessing completo
```bash
python -m preprocessing.process | cat
```

- Output: `models/preprocessed/X.npy`, `models/preprocessed/y.npy`
- Se `preprocess_per_epoch=True`: `models/preprocessed/X_epoch_XXX.npy`, `y_epoch_XXX.npy`

4) Parametri chiave da ricordare
- `session_timeout_seconds`, `window_before_first_malicious_s`, `window_after_first_malicious_s`, `time_bin_seconds`
- `label_propagation.mode`, `noise_filter.*`
- `flow_balance.method`, `flow_balance.ratio`
- `output_mode` (`sequence` vs `mlp_aggregated`)


### Esecuzione training (solo comandi)

1) Seleziona modello
- `TRAINING_CONFIG.model_type`: `gru`, `lstm`, `dense`
- Se `X.npy` è 2D (MLP aggregato), la CLI forza `dense`

2) Avvia training
```bash
python -m training.train | cat
```

- Salvataggi:
  - Modello: `models/best_model.keras`
  - Log configurazioni: `models/training_log.json`
  - Storico loss/accuracy per epoca: `models/training_history.json`

3) Epoche e loss
- Fino a `max_epochs=30`
- Tracciamento `loss`/`val_loss` per analisi convergenza


### Considerazioni progettuali e trade-off

- **Sessionizzazione**: separa conversazioni distinte sullo stesso `Flow_ID` per evitare leakage temporale.
- **N/T finestra**: simula detection realistiche (contesto pre-attacco e persistenza post-detection). Importante per ridurre falsi positivi.
- **Label propagation**: modalità `any` garantisce recall, `majority` bilancia precision/recall, `probabilistic` consente soglia adattiva, `smoothing` robusto a label rumorose.
- **Bilanciamento a flusso**: evitare che pochi attacchi dominino; `undersample` è stabile, `SMOTE` richiede feature aggregate per sessione.
- **Output**: `sequence` utile per RNN/GRU/LSTM; `mlp_aggregated` per MLP rapido con statistiche robuste.
- **Robustezza a label imperfette**: smoothing temporale e majority voting mitigano errori di annotazione.
- **Per-epoca**: shard per allenare in sequenza o parallelo sottinsiemi di flussi.


### Deep dive: Sessionizzazione e reassembly

- Obiettivo: ricostruire conversazioni coerenti gestendo discontinuità temporali sullo stesso `Flow_ID`.
- Algoritmo:
  - Ordina per `Flow_ID` e `Timestamp`.
  - Avvia `Session_Index=0` al primo record; incrementa l'indice quando l'intervallo tra record successivi supera `session_timeout_seconds`.
  - Costruisci `Session_ID = Flow_ID#Session_Index`.
- Proprietà:
  - Complessità lineare O(N) dopo ordinamento.
  - Resiliente a timestamp mancanti: avvia una nuova sessione quando il timestamp non è valido.
- Rischi/Pitfall:
  - Timeout troppo corto frammenta sessioni; troppo lungo fonde conversazioni distinte.
  - Assicurare la normalizzazione del fuso e il parsing coerente dei timestamp (`timestamp_format` se necessario).


### Deep dive: Noise handling (majority & temporal smoothing)

- Majority window: media mobile della variabile binaria `is_malicious`; soglia `threshold` decide la classe.
- Temporal smoothing (EMA): `s_t = alpha * x_t + (1 - alpha) * s_{t-1}` con `alpha` in `(0,1]`.
  - `alpha` alto reagisce più velocemente ma è più sensibile al rumore; `alpha` basso è più stabile.
- Applicazione per sessione: evita bleed-over tra flussi diversi.
- Effetti:
  - Riduce flip sporadici delle etichette.
  - Utile con dataset reali con labeling non perfetto o delay di annotazione.


### Deep dive: Label propagation (any/majority/probabilistica/smoothing)

- any: recall massimo; finestra = malevola se presente QUALSIASI evento malevolo.
- majority: trade-off; richiede >50% eventi malevoli nella finestra.
- probabilistica: imposta soglia `prob_threshold` (es. 0.3, 0.5, 0.7) sulla frazione di eventi malevoli.
- smoothing: usa output filtrato (EMA/majority) e poi majority "soft" per la finestra.
- Scelta operativa:
  - In ambienti ad alto rischio → preferire `any` o soglia bassa.
  - Per ridurre falsi positivi in produzione → `majority` o `smoothing`.


### Deep dive: Finestre N/T e binning temporale

- Strategia `first_malicious_context`:
  - Trova il primo timestamp malevolo nella sessione.
  - Crea una finestra da `N` secondi prima a `T` secondi dopo.
- Binning:
  - Discretizza la finestra in intervalli di `time_bin_seconds` (es. 5s).
  - Aggrega le feature per bin per costruire sequenze temporali regolari.
- Mancanza di eventi in un bin:
  - Il bin risulta zero o con statistiche neutre; il padding finale uniforma le lunghezze.
- Allineamento:
  - Base temporale = minimo timestamp della finestra per robustezza.


### Deep dive: Aggregazioni statistiche e feature engineering

- Statistiche per bin/finestra: `sum`, `mean`, `std`, `min`, `max`.
- Perché:
  - `sum` e `mean` catturano intensità media; `std` quantifica variabilità; `min/max` estremi utili per burst.
- Estensioni possibili (future):
  - Entropia di porte/destinazioni IP per finestra; tassi di errore; inter-arrival variance robusta.
  - Indicatori di burst (peak-to-average ratio), code di coda (kurtosi) per anomalia.


### Bilanciamento a livello di flusso

- Definizione di finestra/Session come unità di campionamento.
- `undersample`: seleziona tutte le sessioni malevole + sottoinsieme di `BENIGN` fino a parità (configurabile via `ratio`).
- `SMOTE`: richiede rappresentazione tabellare per sessione (aggregate), poi genera campioni sintetici nello spazio feature.
- Considerazioni:
  - `SMOTE` su feature aggregate preserva statistiche di finestra ma non la cronologia di pacchetti.
  - Alternativa avanzata: ADASYN o SMOTE-NC per misti cat/num.


### Comandi utili di audit e ispezione artefatti

- Verifica artefatti preprocessing
```bash
ls -lh models/preprocessed | cat
```

- Ispeziona dimensioni dei file NPY (solo comandi)
```bash
for f in models/preprocessed/*.npy; do python -c "import numpy as np, sys; a=np.load(sys.argv[1], allow_pickle=False); print(sys.argv[1], a.shape)" "$f"; done | cat
```

- Installa jq (se mancante) e visualizza log/history
```bash
sudo apt-get update && sudo apt-get install -y jq
jq '.' models/training_log.json | head -n 50 | cat
jq '.' models/training_history.json | head -n 80 | cat
```

- Pulizia artefatti
```bash
rm -f models/preprocessed/*.npy models/best_model.keras models/training_*.json
```


### Training per-epoca con shard

Se `TRAINING_CONFIG.preprocess_per_epoch=True`, il preprocessing salva shard `X_epoch_XXX.npy` e `y_epoch_XXX.npy`.

- Esempio di training sequenziale per shard (solo comandi)
```bash
for xp in models/preprocessed/X_epoch_*.npy; do 
  yp=${xp/X_epoch_/y_epoch_}
  echo "== Training su $xp =="
  # sostituisci i file full con shard correnti
  cp "$xp" models/preprocessed/X.npy
  cp "$yp" models/preprocessed/y.npy
  python -m training.train | cat
done
```

- Esempio di ispezione prestazioni cumulative (placeholder, solo aggregazione file)
```bash
cat models/training_log.json | wc -l
```


dl

# SNN-IDS Audit Notebook (CSE-CIC-IDS2018)

Questo notebook è pensato per essere auditabile: ogni scelta è documentata file-per-file e riga-per-riga dove rilevante. Include:
- Setup ambiente e dati
- Pipeline dati riproducibile
- Training recipe ottimizzata per tabulari (GRU per finestre temporali)
- Metriche e calibrazione
- Smoke test e test completo (finestre: 5s, 1m, 5m; LR grid)

Note: il codice vive nel repository; il notebook chiama le funzioni senza duplicazioni di logica.


## 1) Setup ambiente

Requisiti minimi:
- Python 3.10+
- pacchetti: pandas, numpy, scikit-learn, tensorflow, matplotlib, seaborn, tqdm

In Colab eseguire le celle seguenti; in locale assicurarsi che `pip install -r requirements.txt` sia stato eseguito.


In [ ]:
# Se sei in Colab, decommenta le righe seguenti
!git clone https://github.com/devedale/snn-ids.git
%cd snn-ids
!pip install -q pandas numpy scikit-learn tensorflow matplotlib seaborn tqdm


## 2) Setup dati
Scarica i CSV CSE-CIC-IDS2018 nelle cartelle già attese da `config.py` (`data/cicids/2018`). In Colab puoi caricare dal tuo Drive o usare Kaggle API.


In [ ]:
%cd snn-ids
!curl -L -o ./data/cicids2018.zip  https://www.kaggle.com/api/v1/datasets/download/edoardodalesio/intrusion-detection-evaluation-dataset-cic-ids2018
#!unzip -o ./data/cicids2018.zip  "Tuesday-20-02-2018.csv"   "Wednesday-21-02-2018.csv"  "Thursday-22-02-2018.csv" "Friday-23-02-2018.csv" -d ./data
!unzip -o ./data/cicids2018.zip -d ./data
# Importa librerie del progetto
import os, json
import numpy as np
import pandas as pd
from config import DATA_CONFIG, PREPROCESSING_CONFIG, TRAINING_CONFIG, BENCHMARK_CONFIG
from preprocessing.process import preprocess_pipeline
from training.train import train_model
from evaluation.metrics import evaluate_model_comprehensive

print("Config dataset:", DATA_CONFIG["dataset_path"])


## 3) Smoke test (GRU)
Esegue pipeline ridotta per verificare fine-to-end: bilanciamento security, IP→ottetti, finestre, training GRU (K-Fold), valutazione con PNG.


In [ ]:
!python3 benchmark.py --smoke-test --data-path data

## 4) Test completo (GRU) con finestre 5s, 1m, 5m e grid LR
In questo test variamo:
- finestre temporali: `window_size` e `step` coerenti con risoluzioni 5s, 1m, 5m
- learning rate: `[1e-3, 5e-4, 1e-4]`
- epoche moderate per tempi ragionevoli


In [ ]:
from copy import deepcopy

results = []
base_prep = deepcopy(PREPROCESSING_CONFIG)
base_train = deepcopy(TRAINING_CONFIG)

# Grid finestre (timesteps) e learning rate
window_configs = [
    {"name": "5s", "window_size": 10, "step": 5},
    {"name": "1m", "window_size": 60//6, "step": 10},  # es: 10 step 
    {"name": "5m", "window_size": 50, "step": 10},
]
lr_grid = [1e-3, 5e-4, 1e-4]

for wc in window_configs:
    PREPROCESSING_CONFIG['use_time_windows'] = True
    PREPROCESSING_CONFIG['window_size'] = wc['window_size']
    PREPROCESSING_CONFIG['step'] = wc['step']
    
    for lr in lr_grid:
        TRAINING_CONFIG['model_type'] = 'gru'
        TRAINING_CONFIG['hyperparameters']['epochs'] = [5]
        TRAINING_CONFIG['hyperparameters']['batch_size'] = [64]
        TRAINING_CONFIG['hyperparameters']['learning_rate'] = [lr]
        
        print(f"\n=== Config: {wc['name']} | lr={lr} ===")
        X, y, le = preprocess_pipeline()
        model, log, path = train_model(X, y, model_type='gru')
        X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        rep = evaluate_model_comprehensive(model, X_te, y_te, le.classes_.tolist(), output_dir=f'notebook_eval/{wc["name"]}_lr{lr}')
        results.append({'window': wc['name'], 'lr': lr, 'accuracy': rep['basic_metrics']['accuracy']})

# Ripristina config
PREPROCESSING_CONFIG.update(base_prep)
TRAINING_CONFIG.update(base_train)

pd.DataFrame(results).sort_values('accuracy', ascending=False).head()


## 5) Riproducibilità
Impostiamo i seed per rendere i risultati ripetibili (entro i limiti dell'hardware).


In [ ]:
import os, random
import numpy as np
import tensorflow as tf

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print('Seed impostato:', SEED)


## 6) Audit dati e feature
Controlliamo distribuzione classi, percentuali, e presenza di attacchi rilevanti nel sample selezionato.


In [ ]:
from collections import Counter

def audit_distribution(y, label_encoder):
    counts = Counter(y)
    classes = label_encoder.classes_.tolist()
    dist = {classes[i]: int(counts.get(i, 0)) for i in range(len(classes))}
    total = sum(dist.values())
    df = pd.DataFrame({
        'classe': list(dist.keys()),
        'conteggio': list(dist.values())
    }).sort_values('conteggio', ascending=False)
    df['percentuale'] = (df['conteggio'] / total * 100).round(2)
    return df

# Esempio live (riutilizza X,y,label_encoder se esistono)
try:
    audit_distribution(y, label_encoder)
except Exception as e:
    print('Esegui prima il smoke test per generare X,y,label_encoder')


## 7) Metriche e calibrazione
Oltre alle metriche standard, aggiungiamo ECE (Expected Calibration Error) per valutare la calibrazione delle probabilità.


In [ ]:
import numpy as np

def expected_calibration_error(y_true, y_proba, n_bins=10):
    # binning su max probability
    confidences = y_proba.max(axis=1)
    predictions = y_proba.argmax(axis=1)
    accuracies = (predictions == y_true).astype(float)
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    ece = 0.0
    for i in range(n_bins):
        mask = (confidences > bins[i]) & (confidences <= bins[i+1])
        if mask.any():
            avg_conf = confidences[mask].mean()
            avg_acc = accuracies[mask].mean()
            ece += np.abs(avg_acc - avg_conf) * mask.mean()
    return float(ece)

# Esempio: usa il modello dallo smoke test, se disponibile
try:
    y_proba = model.predict(X_te, verbose=0)
    print('ECE:', expected_calibration_error(y_te, y_proba, n_bins=15))
except Exception as e:
    print('Esegui prima smoke test e valutazione per avere y_te e y_proba')


## 8) Documentazione file-per-file
In questa sezione spieghiamo le scelte implementative nei file chiave: `preprocessing/process.py`, `training/train.py`, `evaluation/metrics.py`, `benchmark.py` e `config.py`.


In [ ]:
import inspect, textwrap
import preprocessing.process as P
import training.train as T
import evaluation.metrics as E
import benchmark as B
import config as C

def show_source(obj, start=None, end=None):
    src = inspect.getsource(obj)
    if start or end:
        lines = src.splitlines()
        src = "\n".join(lines[start:end])
    print(textwrap.dedent(src))

print('--- config.py (sezioni principali) ---')
print('DATA_CONFIG:'); print(C.DATA_CONFIG)
print('\nPREPROCESSING_CONFIG:'); print(C.PREPROCESSING_CONFIG)
print('\nTRAINING_CONFIG:'); print(C.TRAINING_CONFIG)

print('\n--- preprocessing.process: load_and_balance_dataset ---')
show_source(P.load_and_balance_dataset)
print('\n--- preprocessing.process: preprocess_pipeline ---')
show_source(P.preprocess_pipeline)

print('\n--- training.train: _train_k_fold ---')
show_source(T._train_k_fold)
print('\n--- training.train: _train_split ---')
show_source(T._train_split)

print('\n--- evaluation.metrics: evaluate_model_comprehensive ---')
show_source(E.evaluate_model_comprehensive)

print('\n--- benchmark.SNNIDSBenchmark (run_smoke_test) ---')
show_source(B.SNNIDSBenchmark.run_smoke_test)


### Note progettuali
- Zero hard-code: tutte le scelte sono in `config.py`; il notebook applica override solo per esperimenti.
- Pipeline riproducibile: sampling e bilanciamento documentati; seed fissati.
- Training recipe tabulari: GRU su finestre 3D, scaling per-fold, StratifiedKFold.
- Metriche e PNG: confusion matrix dettagliata, cybersecurity, ROC, accuracy per classe, ECE.
- Notebook auditabile: usa `inspect` per mostrare il codice sorgente eseguito.


## 9) Limitazioni
- I risultati su classi rare vanno interpretati con cautela; forniamo sempre breakdown per‑classe.
- La calibrazione (ECE) è informativa ma non esaustiva.
- Il bilanciamento “security” riduce bias ma non sostituisce protocolli di acquisizione realistici.
- Evitiamo leakage scalando per‑fold; ulteriori audit sono comunque consigliati in ambienti operativi.
- Per produzione sono necessarie valutazioni cost‑sensitive e monitoraggio del drift.
